In [1]:
import pandas as pd
import os
from pathlib import Path
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
from torch.optim import adam
from tqdm import tqdm
from sklearn.metrics import balanced_accuracy_score, accuracy_score

current_working_dir = os.getcwd()
GENERAL_DATA_PATH = Path(current_working_dir).parent / 'data'
DATA_PATH = GENERAL_DATA_PATH

In [2]:
train_poses = pd.read_csv(DATA_PATH / 'train-poses.csv', sep=';')
train_labels = pd.read_csv(DATA_PATH / 'train-labels.csv', sep=',')

In [3]:
from sklearn.model_selection import train_test_split

raw_data = pd.merge(train_poses, train_labels, on='ID')
data = raw_data.drop(columns=['ID']).dropna()

encoder = LabelEncoder()
encoder.fit(raw_data['PoseLabel'].values)

data['PoseLabel'] = torch.from_numpy(encoder.transform(data['PoseLabel'].values))

train_data, valid_data = train_test_split(data, test_size=0.3, random_state=42, stratify=data['PoseLabel'])

print(len(train_data), len(valid_data))

8029 3442


In [4]:
len(train_data['PoseLabel'].value_counts(normalize=True))

80

In [5]:
train_classs_weights = torch.tensor(1 / train_data['PoseLabel'].value_counts(normalize=True).sort_index().values, dtype=torch.float32)
print(len(train_classs_weights))

80


In [6]:
class MyDataset(Dataset):
    def __init__(self, data: pd.DataFrame):
        self.data_features = torch.from_numpy(data.drop(columns=['PoseLabel']).values)
        self.data_label_numbers = torch.from_numpy(data['PoseLabel'].values).long()

    def __len__(self):
        return len(self.data_features)

    def __getitem__(self, idx):
        features = self.data_features[idx]
        label = self.data_label_numbers[idx]
        return features, label

train_dataset = MyDataset(train_data)
valid_dataset = MyDataset(valid_data)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)

In [7]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.dense_1 = nn.Sequential(
            nn.Linear(132, 64),
            nn.BatchNorm1d(64),
            nn.Dropout(0.5),
            nn.ReLU()
        )
        self.dense_2 = nn.Sequential(
            nn.Linear(64, 64),
            nn.BatchNorm1d(64),
            nn.Dropout(0.5),
            nn.ReLU()
        )
        self.classification_head = nn.Sequential(
            nn.Linear(64, 80)
        )

    def forward(self, x):
        x = self.dense_1(x)
        x = x + self.dense_2(x)
        x = self.classification_head(x)
        return x

In [8]:

def train(model, train_loader, optimizer, criterion, epochs=100):
    model.train()
    train_loss_logs = []
    train_loss = 0
    
    for epoch in tqdm(range(epochs)):
        for batch, (features, labels) in enumerate(train_loader):
            optimizer.zero_grad()
            output = model(features.float())
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            train_loss_logs.append(loss.item())
        if epoch % 10 == 0:
            print(f'Epoch {epoch}, Loss {loss.item()}')
    print(f'Epoch {epoch}, Loss {loss.item()}')
    
    return train_loss_logs

def validate(model, valid_loader, criterion):
    model.eval()
    valid_loss = 0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for batch, (features, labels) in enumerate(valid_loader):
            output = model(features.float())
            loss = criterion(output, labels)
            valid_loss += loss.item()
            _, predicted = output.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    
    accuracy = accuracy_score(labels, predicted)
    balanced_accuracy = balanced_accuracy_score(labels, predicted)
    
    return valid_loss / len(valid_loader), accuracy, balanced_accuracy

def predict(model, data_loader):
    model.eval()
    predictions = []
    with torch.no_grad():
        for batch, features in enumerate(data_loader):
            output = model(features.float())
            _, predicted = output.max(1)
            predictions.extend([x.item() for x in predicted])
    return predictions

In [9]:
net = Net()
optimizer = adam.Adam(net.parameters(), lr=1e-3, weight_decay=1e-5)
criterion = nn.CrossEntropyLoss(weight=train_classs_weights)
logs = train(model=net, train_loader=train_loader, optimizer=optimizer, criterion=criterion)

  1%|          | 1/100 [00:01<02:44,  1.66s/it]

Epoch 0, Loss 3.554771900177002


 11%|█         | 11/100 [00:12<01:32,  1.04s/it]

Epoch 10, Loss 2.3701117038726807


 21%|██        | 21/100 [00:22<01:22,  1.05s/it]

...

Epoch 80, Loss 0.9153197407722473


 91%|█████████ | 91/100 [01:33<00:09,  1.00s/it]

Epoch 90, Loss 1.1956923007965088


100%|██████████| 100/100 [01:42<00:00,  1.02s/it]

Epoch 99, Loss 1.5674970149993896


In [10]:
validate(model=net, valid_loader=valid_loader, criterion=criterion)

d:\repos\knum-2024-pose-estimation\pe-venv\Lib\site-packages\sklearn\metrics\_classification.py:2458: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


(0.8000130923809828, 0.7222222222222222, 0.7058823529411765)

In [11]:
full_train_classs_weights = torch.tensor(1 / data['PoseLabel'].value_counts(normalize=True).sort_index().values, dtype=torch.float32)
print(len(full_train_classs_weights))

80


In [12]:
full_train_dataset = MyDataset(data)
full_train_loader = DataLoader(full_train_dataset, batch_size=32, shuffle=True)

In [13]:
net_full_train = Net()
optimizer = adam.Adam(net_full_train.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss(weight=full_train_classs_weights)
logs = train(model=net_full_train, train_loader=full_train_loader, optimizer=optimizer, criterion=criterion, epochs=100)

  1%|          | 1/100 [00:02<03:46,  2.29s/it]

Epoch 0, Loss 3.215073347091675


 11%|█         | 11/100 [00:20<02:57,  2.00s/it]

Epoch 10, Loss 1.820868968963623


 21%|██        | 21/100 [00:36<02:11,  1.66s/it]

...

Epoch 80, Loss 1.3478281497955322


 91%|█████████ | 91/100 [02:13<00:11,  1.28s/it]

Epoch 90, Loss 2.1970512866973877


100%|██████████| 100/100 [02:23<00:00,  1.44s/it]

Epoch 99, Loss 1.7390457391738892


In [14]:
test_poses = pd.read_csv(DATA_PATH / 'test-poses.csv', sep=';')
test_labels = pd.read_csv(DATA_PATH / 'test-labels.csv', sep=',')
test_data = pd.merge(test_poses, test_labels, on='ID').drop(columns=['ID']).dropna()
test_poses_cleaned = test_poses.drop(columns=['ID']).dropna()

class MyNoLabelsDataset(Dataset):
    def __init__(self, data):
        self.data = torch.from_numpy(data.values)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        sample = self.data[index]
        return sample

test_dataset = MyNoLabelsDataset(test_poses_cleaned)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [15]:
print(len(raw_data), len(data), len(raw_data) - len(data))
print(len(test_poses), len(test_poses_cleaned), len(test_poses) - len(test_poses_cleaned))

12690 11471 1219
4232 3835 397


In [16]:
preds = predict(net_full_train, test_loader)

In [17]:
decoded_preds = encoder.inverse_transform(preds)

In [18]:
argmax_nondetected_label = raw_data[raw_data.isnull().any(axis=1)][['PoseLabel']].value_counts().idxmax()[0]

In [30]:
submission = test_poses.dropna()[['ID']]
submission['PoseLabel'] = decoded_preds

submission = pd.merge(submission, test_labels, on='ID', how='outer').fillna(argmax_nondetected_label)
len(submission)

4232

In [32]:
submission.to_csv('submission.csv', index=False)